In [3]:
import pandas as pd
import numpy as np
from keras.layers import Dense, LSTM, Dropout, Activation, Embedding, Flatten, MaxPool2D, UpSampling2D
from keras.layers import Conv2D, Reshape, Conv1D
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import math
import matplotlib.pyplot as plt
from keras.initializers import Constant, glorot_normal

Using TensorFlow backend.


In [4]:
from keras.datasets import mnist

In [5]:
def newNode(units, hidden, input_shape ):
    model = Sequential()
    model.add(Dense(units = hidden, input_shape = input_shape, activation = 'sigmoid'))
    model.compile(loss='mse', optimizer = 'adagrad', metrics = ['accuracy'])
    modelx = Sequential()
    modelx.add(model)
    modelx.add(Dense(units = units, activation = 'sigmoid'))
    modelx.compile(loss='mse', optimizer = 'adagrad', metrics = ['accuracy'])
    return modelx, model



def getDecisionMaker( input_shape ):
    model = Sequential()
    model.add(Dense(units = 1, input_shape = input_shape,
                    activation = 'sigmoid', 
                    kernel_initializer=glorot_normal(seed=0.1)))
    model.compile(loss='mse', optimizer = 'adagrad', metrics = ['accuracy'])
    model.trainable = False
    for l in model.layers:
        l.trainable = False
        
    return model
    

In [45]:
model_count = 0
cluster_count = 10
dm_input = 256
output_len = 10
dm = getDecisionMaker((dm_input,))
dm.save_weights('dmconstant')

In [7]:
images = mnist.load_data()

In [26]:
train, test = images
test_x, test_y = test
train_x, train_y = train
test_x = test_x.reshape(10000, 784)
train_x = train_x.reshape(60000, 784)
train_y = to_categorical(train_y, num_classes=10)
test_y = to_categorical(test_y, num_classes=10)

In [46]:
t, p = newNode(output_len, 256, (784,))
nodes = [newNode(output_len, dm_input,input_shape=(dm_input,)) for i in range(cluster_count)]

In [41]:
def mkd(a, b):
    if a > b:
        return 0, a
    return 1, b

from collections import defaultdict

In [51]:
def make_trainable(model, v = True):
    model.trainable = v
    for l in model.layers:
        l.trainable = v

In [53]:
t.fit(train_x, train_y, epochs=1, verbose = 1)
make_trainable(t, False)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.0061 - acc: 0.9674    


In [54]:
newx = p.predict(train_x)

In [78]:
def make_seperate_samples(xs, ys):
    tmp = defaultdict(lambda:[np.array([]), np.array([])])
    count = 0
    for x, y in zip(xs, ys):
        x = np.array([x])
        index= int(dm.predict(x) * cluster_count)
        tmp[index][0] = np.append(tmp[index][0], x)
        tmp[index][1] = np.append(tmp[index][1], y)
        count += 1
        print("{:10d}".format(count), end='\r')
    return tmp

data = make_seperate_samples(newx, train_y)

In [79]:
data

defaultdict(<function __main__.make_seperate_samples.<locals>.<lambda>>,
            {1: [array([ 1.,  0.,  0., ...,  1.,  0.,  0.]),
              array([ 1.,  0.,  0., ...,  0.,  0.,  0.])],
             2: [array([ 1.,  1.,  0., ...,  0.,  0.,  0.]),
              array([ 1.,  0.,  0., ...,  0.,  0.,  0.])],
             3: [array([  1.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
                       7.20740431e-23,   0.00000000e+00,   0.00000000e+00]),
              array([ 0.,  0.,  1., ...,  0.,  0.,  0.])],
             4: [array([ 1.,  0.,  1., ...,  1.,  0.,  0.]),
              array([ 0.,  0.,  0., ...,  0.,  1.,  0.])],
             5: [array([ 1.,  0.,  0., ...,  0.,  0.,  0.]),
              array([ 0.,  0.,  0., ...,  0.,  0.,  0.])],
             6: [array([ 1.,  0.,  0., ...,  1.,  0.,  1.]),
              array([ 0.,  0.,  0., ...,  0.,  0.,  0.])],
             7: [array([  1.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
                       1.

In [93]:
for i, (x, y) in data.items():
    x = x.reshape((len(x)//256, 256))
    y = y.reshape((len(y)//10, 10))
    nodes[i][0].fit(x, y, epochs = 1, verbose = 1)

Epoch 1/1
207/207 [==============================] - 0s - loss: 0.0064 - acc: 0.9565     
Epoch 1/1
4835/4835 [==============================] - 0s - loss: 0.0042 - acc: 0.9760     
Epoch 1/1
10728/10728 [==============================] - 1s - loss: 0.0050 - acc: 0.9724     
Epoch 1/1
17084/17084 [==============================] - 1s - loss: 0.0049 - acc: 0.9717     
Epoch 1/1
15530/15530 [==============================] - 1s - loss: 0.0049 - acc: 0.9729     
Epoch 1/1
8524/8524 [==============================] - 1s - loss: 0.0049 - acc: 0.9715     
Epoch 1/1
2964/2964 [==============================] - 0s - loss: 0.0041 - acc: 0.9784     
Epoch 1/1
128/128 [==============================] - 0s - loss: 0.0057 - acc: 0.9688     


In [64]:
for _ in range(10):
    accuracy = defaultdict(float)
    counts = defaultdict(int)
    count = 1
    totl = 0
    tota = 0
    ll, rr = 0, 0
    for x, y in zip(newx[:20000], train_y):
        x = np.array([x])
        y = np.array([y])
        index= int(dm.predict(x) * cluster_count)
        h = nodes[index][0].fit(x, y, epochs = 1, verbose = 0)
        loss, acc = (h.history['loss'][0],h.history['acc'][0])
        accuracy[index] += acc
        counts[index] += 1
        print("epoch : {:7d}, loss: {:.6f}, acc: {:.6f}, {}:{}".format(
            count,totl/ count,tota/ count, index,accuracy[index]/ counts[index]), end = '\r')
        count += 1
    t = {}
    for i, e in accuracy.items():
        t[i] = e / counts[i]
    print(t)
        
        
        

{1: 1.0, 2: 0.97708978328173379, 3: 0.9670177383592018, 4: 0.97538407821229045, 5: 0.9696442382057231, 6: 0.97241867043847241, 7: 0.98303287380699889, 8: 1.0}
{1: 1.0, 2: 0.97708978328173379, 3: 0.96784922394678496, 4: 0.97590782122905029, 5: 0.96983758700696054, 6: 0.97277227722772275, 7: 0.98303287380699889, 8: 1.0}
{1: 1.0, 2: 0.97832817337461297, 3: 0.96784922394678496, 4: 0.97625698324022347, 5: 0.97022428460943544, 6: 0.97277227722772275, 7: 0.98303287380699889, 8: 1.0}
{1: 1.0, 2: 0.97832817337461297, 3: 0.96812638580931265, 4: 0.97643156424581001, 5: 0.97061098221191033, 6: 0.97277227722772275, 7: 0.98303287380699889, 8: 1.0}
{1: 1.0, 2: 0.97832817337461297, 3: 0.96895787139689582, 4: 0.97660614525139666, 5: 0.97099767981438512, 6: 0.97312588401697309, 7: 0.98409331919406151, 8: 1.0}
{1: 1.0, 2: 0.97832817337461297, 3: 0.96978935698447899, 4: 0.97695530726256985, 5: 0.97157772621809746, 6: 0.97347949080622342, 7: 0.98409331919406151, 8: 1.0}
{1: 1.0, 2: 0.97894736842105268, 3: 

In [65]:
xx = np.array([])
xx.